In [1]:
import pandas as pd
from sqlalchemy import create_engine


In [2]:
dfm = pd.read_csv("../scripts/data/dublin_meta_3lines.csv")
dfm = dfm[dfm.columns[1:]]

In [10]:
dfm[["time_frame", "first_instant", "last_instant"]] = dfm[["time_frame", "first_instant", "last_instant"]].astype("datetime64[ns]")

In [11]:
df = pd.read_csv("../scripts/data/dublin_points_3lines.csv")
df = df[df.columns[1:]]
df.columns = ['trajectory_id', 'instant', 'lng', 'lat', 'delta_dist',
       'delta_time', 'cum_dist', 'cum_time', 'bearing', 'speed',
       'acceleration', 'turn']

In [13]:
df["instant"] = df["instant"].astype("datetime64[ns]")

In [14]:
dfm.dtypes

trajectory_id                  int64
journey_id                    object
time_frame            datetime64[ns]
vehicle_journey_id             int64
operator                      object
vehicle_id                     int64
first_lat                    float64
first_lng                    float64
last_lat                     float64
last_lng                     float64
first_instant         datetime64[ns]
last_instant          datetime64[ns]
dtype: object

In [15]:
import psycopg2 as ps

TABLEMETA = "dublinmeta_v2"
TABLEPOINTS = "dublinpoints_v2"
IP = "172.22.0.6"
conn = ps.connect(dbname="app",
                  user="postgres",
                  password="monografia",
                  host=IP)

In [16]:
cur = conn.cursor()

cur.execute(f"CREATE TABLE {TABLEMETA} (\
    trajectory_id int PRIMARY KEY, journey_id varchar,\
    time_frame timestamp, vehicle_journey_id int,\
    operator varchar, vehicle_id int,\
    first_lat numeric, first_lng numeric,\
    last_lat numeric, last_lng numeric,\
    first_instant timestamp, last_instant timestamp);")
conn.commit()

In [17]:

engine = create_engine(f'postgresql://postgres:monografia@{IP}/app')

dfm.to_sql(
    TABLEMETA,
    engine,
    if_exists="append",
    index=False,
)


In [18]:
df.dtypes

trajectory_id             int64
instant          datetime64[ns]
lng                     float64
lat                     float64
delta_dist              float64
delta_time                int64
cum_dist                float64
cum_time                  int64
bearing                 float64
speed                   float64
acceleration            float64
turn                     object
dtype: object

In [19]:
df.head()

,trajectory_id,instant,lng,lat,delta_dist,delta_time,cum_dist,cum_time,bearing,speed,acceleration,turn
0,1,2013-01-01 12:00:44,-6.262341,53.391186,19.797672,20,19.797672,20,5.026563,0.989884,0.049494,AFTERNOON
1,1,2013-01-01 12:01:05,-6.257115,53.391460,349.013369,21,368.811041,41,351.695216,16.619684,0.791414,AFTERNOON
2,1,2013-01-01 12:01:25,-6.252785,53.391083,291.108874,20,659.919915,61,345.022590,14.555444,0.727772,AFTERNOON
3,1,2013-01-01 12:01:46,-6.249363,53.390537,235.633843,21,895.553758,82,302.890149,11.220659,0.534317,AFTERNOON
4,1,2013-01-01 12:02:04,-6.248399,53.389648,117.908784,18,1013.462542,100,5.939236,6.550488,0.363916,AFTERNOON


In [20]:
cur = conn.cursor()

cur.execute(f"CREATE TABLE {TABLEPOINTS} (\
    id SERIAL PRIMARY KEY, trajectory_id int, instant timestamp, lat numeric, lng numeric,\
    bearing numeric, speed numeric, acceleration numeric,\
    delta_dist numeric, delta_time numeric,\
    cum_dist numeric, cum_time int,\
    turn varchar);")

conn.commit()

In [21]:
engine = create_engine(f'postgresql://postgres:monografia@{IP}/app')

df.to_sql(
    TABLEPOINTS,
    engine,
    if_exists="append",
    index=False,
)